In [ ]:
import numpy as np
import pandas as pd
import os
import shutil 
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
import os,cv2
import numpy as np
import matplotlib.pyplot as plt
from keras import backend as K
from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization
import keras
from keras.models import Sequential
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
%matplotlib inline

In [ ]:
# Define data path
PATH = os.getcwd()
data_path = PATH + '/CT'
data_dir_list = os.listdir(data_path)

In [ ]:
IMG_WIDTH=224
IMG_HEIGHT=224
num_channel=1
num_epoch=20
IMG_DIM = (IMG_WIDTH, IMG_HEIGHT)
# Define the number of classes
num_classes = 2
labels_name={'Covid':0,'non-Covid':1}

img_data_list=[]
labels_list = []
moment_list = []
moment_list2 = []

for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    label = labels_name[dataset]
    for img in img_list:
        input_img = cv2.imread(data_path + '/'+ dataset + '/'+ img )
        input_img_gray = cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
        
        input_img_resize = cv2.resize(input_img,(224,224))
        input_img_resize_gray =cv2.resize(input_img_gray,(224,224))
        
        moments = cv2.moments(input_img_resize_gray)
        huMoments = cv2.HuMoments(moments)   
        moment_list.append(huMoments)
        
        img_data_list.append(input_img_resize)
        labels_list.append(label)  


In [ ]:
## Image normalization ##
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data /= 255
moment_data = np.array(moment_list)
moment_data = moment_data.astype('float32')
moment_data /= 255


In [ ]:
## Label one hot encoding ##
labels = np.array(labels_list)
print(np.unique(labels,return_counts=True))
Y = np_utils.to_categorical(labels, num_classes)

In [ ]:
## Colour channel expansion ##
if num_channel==1:
    if K.common.image_dim_ordering()=='th':
        moment_data= np.expand_dims(moment_data, axis=1) 
    else:
        moment_data= np.expand_dims(moment_data, axis=3) 
else:
    if K.image_dim_ordering()=='th':
        img_data=np.rollaxis(img_data,3,1)
        print (img_data.shape)

In [ ]:
#Shuffle the dataset
x,y = shuffle(img_data,Y, random_state=2)
x2,y2 = shuffle(moment_data,Y, random_state=2)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=2)
X_train2, X_test2, y_train2, y_test2 = train_test_split(x2, y2, test_size=0.1, random_state=2)


In [ ]:
## Initialization of pre-trained ResNet50
from keras.applications import ResNet50

# include top should be False to remove the softmax layer
pretrained_model = ResNet50(include_top=False, weights='imagenet')

output = pretrained_model.layers[-1].output
output = keras.layers.GlobalAveragePooling2D()(output)
pretrained_model = keras.Model(pretrained_model.input, output)
pretrained_model.summary()

In [ ]:
## Features extraction ##
from keras.utils import to_categorical

vgg_features_train = pretrained_model.predict(X_train)
vgg_features_val = pretrained_model.predict(X_test)

# vgg_features_train = vgg_features_train.reshape(2052,1024)
# vgg_features_val = vgg_features_val.reshape(228, 1024)

In [ ]:
# Reshape moment invariant (training) values into 1 dimension
X_train2_1 = X_train2
Moment_Inv = X_train2_1.reshape(2052,7)
Resnet_MI = K.concatenate([vgg_features_train, Moment_Inv],axis=1)

# Reshape moment invariant (testing) values into 1 dimension
X_test2_1 = X_test2
Moment_Inv_val = X_test2_1.reshape(228,7)
Resnet_MI_val = K.concatenate([vgg_features_val, Moment_Inv_val],axis=1)



In [ ]:
import tensorflow as tf
proto_tensor = tf.make_tensor_proto(Resnet_MI)
input_array = tf.make_ndarray(proto_tensor)
input_array = input_array.reshape(2052, 2055,1)
input_array_shape = input_array[0].shape
print(input_array_shape.shape)

proto_tensor_val = tf.make_tensor_proto(Resnet_MI_val)
input_array_val = tf.make_ndarray(proto_tensor_val)
input_array_val = input_array_val.reshape(228, 2055,1)



In [ ]:
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, InputLayer, BatchNormalization, Dropout, GlobalAveragePooling2D, Activation

# Fully Connected layer
model_fusion = Sequential()
model_fusion.add(Flatten(input_shape=(2055,1)))
model_fusion.add(Dense(256, activation='relu'))
model_fusion.add(BatchNormalization())
model_fusion.add(Dropout(0.5))
model_fusion.add(Dense(2, activation='sigmoid'))


# sgd = SGD(lr=0.01, decay= 1e-8, momentum= 0.4, nesterov=True)
# model_fusion.compile(loss='binary_crossentropy', optimizer=sgd, metrics=["accuracy"])

# sgd = SGD(lr=0.00004, decay= 1e-7, momentum= 0.4, nesterov=True)
# model_fusion.compile(loss='binary_crossentropy', optimizer=sgd, metrics=["accuracy"])
anne = ReduceLROnPlateau(monitor='val_accuracy', factor=0.3, patience=5, verbose=1, min_lr=1e-3)
checkpoint = ModelCheckpoint('model/ResNet50_Moment_Classifier2.h5', verbose=1, save_best_only=True)

model_fusion.compile(loss='binary_crossentropy', optimizer=keras.optimizers.adam(lr=0.00005, decay= 1e-5), metrics=['accuracy'])
model_fusion.summary()
Fusion = model_fusion.fit(input_array, y_train, batch_size = 128 , epochs=100, verbose=1, validation_data=(input_array_val, y_test),callbacks=[anne, checkpoint])

In [ ]:
## Saving training acc and loss to CSV
hist_df = pd.DataFrame(hist.history) 

hist_csv_file = 'history/ResNet_MI_history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)
    
history = pd.read_csv("history/ResNet_MI_history.csv")
val_loss = history['val_loss'].to_numpy()
val_acc = history['val_accuracy'].to_numpy()
loss = history['loss'].to_numpy()
acc = history['accuracy'].to_numpy()

In [ ]:
## Plot Acc vs Loss ##
plt.plot(acc)
plt.plot(loss)
plt.plot(val_acc)
plt.plot(val_loss)
plt.title('Training Accuracy vs Epochs ', fontsize=20)
plt.yticks(fontsize=20)
plt.ylabel('Accuracy (%)', fontsize=20)
plt.xticks(fontsize=20)
plt.xlabel('Epoch',fontsize=20)
plt.legend(['Train_acc', 'Train_loss', 'Val_acc', 'Val_loss'], loc='best', fontsize=22)
plt.gcf().set_size_inches(12,10) 
# plt.savefig('C:/Users/CooL/Desktop/Result/All Training vs Loss/ResNet_MI_train_graph.jpg')
plt.show()

In [ ]:
### Testing phase ###

PATH = os.getcwd()
data_path = PATH + '/Testing_Set'
data_dir_list = os.listdir(data_path)

import seaborn as sns
import matplotlib.pyplot as plt   
from sklearn.metrics import confusion_matrix

test_list = []
moment_test_list = []
moment_test_list2 = []
y_pred = []
labels_name ={'Covid':0,'non-Covid':1}



for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    label = labels_name[dataset]
    for img in img_list:
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
        input_img_resize=cv2.resize(input_img,(224,224))
        
        input_img_moment =cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
        input_img_resize_gray =cv2.resize(input_img_moment,(224,224))
       
        moments = cv2.moments(input_img_resize_gray)
        huMoments = cv2.HuMoments(moments)  
        moment_test_list.append(huMoments)
             
        test_list.append(input_img_resize)
        y_pred.append(label)

test_image = np.array(test_list)
test_image = test_image.astype('float32')
test_image /= 255
moment_data = np.array(moment_test_list)
moment_data = moment_data.astype('float32')
moment_data /= 255


VGG_features_test = pretrained_model.predict(test_image)
VGG_features_test = VGG_features_test.reshape(200,2048)
#predict = np.argmax(Resnet_features_test,axis=1)


# Reshape moment invariant (testing) values into 1 dimension
test_data = moment_data
test_data = test_data.reshape(200,7)
Test_MI = K.concatenate([VGG_features_test, test_data],axis=1)


proto_tensor = tf.make_tensor_proto(Test_MI)
input_array_test = tf.make_ndarray(proto_tensor)
print(input_array_test.shape)
input_array_test = input_array_test.reshape(200, 2055, 1)

predict = model_fusion.predict(input_array_test)
predict = np.argmax(predict,axis=1)

cf_matrix = confusion_matrix(y_pred, predict)
tp = cf_matrix[0][0]
fn = cf_matrix[0][1]
fp = cf_matrix[1][0]
tn = cf_matrix[1][1]

TrueP = '{0:.2%}'.format(tp/(tp+fn))
FalseN = '{0:.2%}'.format(fn/(tp+fn))
FalseP = '{0:.2%}'.format(fp/(fp+tn))
TrueN = '{0:.2%}'.format(tn/(fp+tn))
print(TrueP, FalseN, FalseP, TrueN)

xy_Labels = ['Covid', 'non-Covid']
group_names = ['True Positive', 'False Negative', 'False Positive','True Negative']
group_counts = ['{0:0.0f}'.format(value) for value in cf_matrix.flatten()]
group_percentage = [TrueP, FalseN, FalseP, TrueN]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentage)]
labels = np.asarray(labels).reshape(2,2)
 
### Confusion matrix (ResNet50 + MI) ###
plt.title('(Moment Invariant + ResNet-50)' , fontsize = 18)
sns.heatmap(cf_matrix, fmt='', annot=labels, annot_kws={'size':18} ,xticklabels=xy_Labels, yticklabels=xy_Labels, cmap='PuBuGn')
plt.xlabel('Predicted label', fontsize = 15) # x-axis label with fontsize 15
plt.ylabel('True label', fontsize = 15) # y-axis label with fontsize 15
plt.savefig('C:/Users/CooL/Desktop/Result/ResNet_Moment Invariant/Resnet_Mi_CF.jpg')
plt.show()

from sklearn.metrics import auc, roc_curve
import pandas as pd
predict = model_fusion.predict(input_array_test)
fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_pred, predict[:,1])


auc_keras = auc(fpr_keras, tpr_keras)
#print("FPR: ",fpr_keras,"TPR: ", tpr_keras,"Threshold: ", thresholds_keras)
optimal = thresholds_keras[np.argmax(tpr_keras - fpr_keras)]
print('Optimal cut off point = ',optimal)
print('AUC = ', auc_keras)

### ROC Curves (ResNet50 + MI) ###
plt.figure(figsize=(6,6))
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
plt.xlabel('False positive rate', fontsize=15)
plt.ylabel('True positive rate', fontsize=15)
plt.title('ROC (ResNet-50 + Moment Invariant)',fontsize=22)
plt.legend(loc='best', fontsize=15)
plt.savefig('C:/Users/CooL/Desktop/Result/ResNet_Moment Invariant/Resnet_MIgraph.jpg')
plt.show()
